# Session 5

1. Combiner les données
2. Stratégie split-apply-combine (groupby)
3. Index hiérarchiques
4. Méthodes de reshaping (2)

In [ ]:
# imports
import numpy as np
import pandas as pd

**Dataset n° 1 et n° 2**

The GeoNames geographical database covers all countries and contains over eleven million placenames that are available for download free of charge.

https://www.geonames.org/

#### Pays

Voir : https://www.geonames.org/countries/

In [ ]:
# dataset 1
var = pd.read_html('https://www.geonames.org/countries/',
                   header=0,
                   keep_default_na=False,  # NA = North America
                   encoding="utf-8"
                  )
[x.shape for x in var]

In [ ]:
# pays
df_pays = var[1]
df_pays.head()

In [ ]:
# nombre de pays par continent
df_pays['Continent'].value_counts()

#### Villes

Voir : http://download.geonames.org/export/dump/

In [ ]:
# villes
df = pd.read_csv('cities500.zip',
                 sep='\t',
                 header=None,
                 keep_default_na=False,  # NA = North America
                 na_values=['', -9999],
                 names=['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 
                        'longitude', 'feature class', 'feature code', 'country code', 
                        'cc2', 'admin1 code', 'admin2 code', 'admin3 code', 'admin4 code', 
                        'population', 'elevation', 'dem', 'timezone', 'modification date'],
                dtype={'admin1 code': str,
                       'admin2 code': str,
                       'admin3 code': str,
                       'admin4 code': str})
df.shape

In [ ]:
# villes
df.head()

In [ ]:
# nombre de villes par pays
df['country code'].value_counts().head(16)

**Exercice n° 1**

- Quelle est la ville la plus peuplée (population) ?
- Quelle est la ville la plus haute (elevation ou dem) ?
- Quelle est la ville la plus basse (elevation ou dem) ?
- La colonne "alternatenames" contient pour chaque ville les différents noms de chacune des villes, séparés par des virgules. Quelle ville possède le plus de noms différents ? Donnez la liste des noms.

### 1. Combiner les données

Union ensembliste :
- fonction `concat()` : concaténation de Series ou DataFrames à partir d'une liste
- <strike>à noter la méthode `append()` est à présent *deprecated*</strike>

Jointure de bases de données :
- fonction `merge()` : jointure de 2 DataFrames ('on' ou bien 'left_on' + 'right_on', 'how', 'suffixes' pour les colonnes dupliquées)
- méthode `join()` : jointure d'un DataFrame à un autre ('on', 'how', 'rsuffix', 'lsuffix' pour les colonnes dupliquées)


`pd.merge(df1, df2) <=> df1.join(df2)`



Mot-clé 'how' :
- 'inner' (SQL INNER JOIN) : intersection des valeurs des 2 colonnes de jointure
- 'left' (SQL LEFT OUTER JOIN) : valeurs de la colonne de jointure de gauche
- 'right' (SQL RIGHT OUTER JOIN) : valeurs de la colonne de jointure de droite
- 'outer' (SQL OUTER JOIN) : union des valeurs des 2 colonnes de jointure
- 'cross' (SQL CROSS JOIN) : produit cartésien des 2 colonnes de jointure

Voir : https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [ ]:
# merge df et df_pays avec le code iso2
df = pd.merge(df,
              df_pays,
              left_on='country code',
              right_on='ISO-3166alpha2',
              how='left')
df.to_pickle('df_geo.pkl')
df.head()

In [ ]:
pd.read_pickle('df_geo.pkl')

In [ ]:
df.info()

In [ ]:
# vérification de l'identité des 2 colonnes (INUTILE !)
(df['country code'] == df['ISO-3166alpha2']).all()

In [ ]:
# nombre de villes par continent
df['Continent'].value_counts()

### 2. Stratégie split-apply-combine

La stratégie split-apply-combine consiste à :
- éclater les données en sous-groupes sur la base d'un critère (par ex., les valeurs d'une colonne)
- appliquer une fonction à chacun des sous-groupes indépendamment
- combiner les résultats en une structure de données

`df.groupby()` retourne un objet de type DataFrameGroupBy qui peut être vu comme un dictionnaire dont les clés sont les différentes valeurs de la colonne utilisée pour éclater les données, et dont les valeurs sont des sous-DataFrames ou des sous-Series correspondant aux données éclatées.

Voir : https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

In [ ]:
# groupby Continent
cont_group = df.groupby('Continent')
type(cont_group)

In [ ]:
# ngroups
cont_group.ngroups

In [ ]:
# sous-groupe
cont_group.get_group('AN')

### Application de méthodes standard

- `size()` : group sizes
- `count()` : count of group
- `mean()` : mean of groups
- `sum()` : sum of group values
- `std()` : standard deviation of groups
- `var()` : variance of groups
- `first()` : first of group values
- `last()` : last of group values
- `nth()` : nth value
- `min()` : min of group values
- `max()` : max of group values

In [ ]:
# exemple avec size()
df.groupby('Continent').size()

In [ ]:
# exemple avec size()
df.groupby('Country').size()

In [ ]:
# exemple avec sum()
df.groupby('Continent')['population'].sum()

In [ ]:
# exemple avec sum()
df.groupby('Continent').sum()

### Méthode aggregate() ou agg()

Applique une fonction, une liste de fonctions ou un dictionnaire de fonctions à un groupby.

#### Avec une fonction simple

In [ ]:
# exemple
df.groupby('Continent').agg('mean')  # idem que df.groupby('Continent').mean()

#### Avec une liste de fonctions

In [ ]:
# exemple
df.groupby('Continent').agg(['mean', 'std'])

In [ ]:
# exemple
df.set_index('Continent').select_dtypes(include=np.number).reset_index().groupby('Continent').agg(['mean', 'std'])

#### Avec un dictionnaire de fonctions

In [ ]:
# exemple avec un dictionnaire
df.groupby('Continent').agg({'population': 'sum',
                             'elevation': 'mean',
                             'Country': ['min', 'max']})

### Méthode apply()

Applique une lambda ou une fonction définie à un groupby.

In [ ]:
df.info()

In [ ]:
# retourne la ville la plus peuplée d'un DataFrame
def top1city(group):
    return group.loc[group['population'].idxmax(), 'name']

In [ ]:
# avec df
top1city(df)

In [ ]:
# avec le groupby Continent
df.groupby('Continent').apply(top1city)

In [ ]:
# avec le groupby Country
df.groupby('Country').apply(top1city)

In [ ]:
# retourne les 3 villes les plus peuplées d'un DataFrame
def top3city(group):
    return group.nlargest(3, 'population')['name']

In [ ]:
# avec df
top3city(df)

In [ ]:
# par continent
df.groupby('Continent').apply(top3city).droplevel(1)  # supprime le multi-index

**Exercice n° 2**

Créez une fonction qui calcule la moyenne du nombre de noms alternatifs de chaque ville.

Testez-la sur tout le dataset.

Appliquez cette fonction sur un groupby 'feature code'. Quel code obtient la valeur la plus grande ?

Voir les features codes des pays : https://www.geonames.org/export/codes.html#P

### 3. Index hiérarchiques

pandas est capable de gérer des index hiérarchiques.

Voir : https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html

#### 3.1 Index

In [ ]:
# groupby 2 colonnes
# nombre de villes par pays et par continent
s = df.groupby(['Continent', 'Country']).size()
s

In [ ]:
# type de l'index
type(s.index)

In [ ]:
# niveaux de l'index : Continent, Country
s.index.levels

In [ ]:
# accès au premier niveau
s.loc['AF'] # ou s.loc[('AF',)]

In [ ]:
# accès au second niveau
s.loc[('AF', 'Algeria')]

In [ ]:
# reset Continent
s.reset_index('Continent') # ou s.reset_index(level=0)

In [ ]:
# reset Country
s.reset_index('Country') # ou s.reset_index(level=1)

In [ ]:
# reset Country
s.reset_index('Country').reset_index() # ou s.reset_index(level=1)

In [ ]:
# swaplevel
s.swaplevel()

In [ ]:
# droplevel
s.droplevel(1)

#### 3.2 Columns

In [ ]:
# retour sur un dataframe avec des colonnes hiérarchiques
tab = df.groupby('Continent').agg({'population': 'sum',
                                   'elevation': 'mean',
                                   'Country': ['min', 'max']})
tab

In [ ]:
# columns
tab.columns

In [ ]:
# accès au premier niveau
tab.loc[:, 'Country']  # ou tab.loc[:, ('Country',)]

In [ ]:
# accès au second niveau
tab.loc[:, ('Country', 'min')]

In [ ]:
# swaplevel
tab.swaplevel(axis=1)

In [ ]:
# droplevel
tab.droplevel(0, axis=1)

### 4. Méthodes de reshaping (2)

**pandas** possède plusieurs méthodes de reshaping qui généralisent les méthodes de pivot :
- `stack()` : move the inner-most (or the specified) column level to the inner-most index level
- `unstack()` : move the inner-most (or the specified) index level to the inner-most column level
- `swaplevel()` : swap 2 levels from index or from columns
- `droplevel()` : drop a level of an index or of a column

D'autres méthodes permettent de manipuler les index et peuvent être combinées :
- `set_index()` : move the specified column as the new index
- `reset_index()` : move the specified index as a new column
- `reindex()` : conform to a new index

Enfin, la méthode `melt()` permet de faire passer une table d'un format large vers un format long.

Toutes ces méthodes sont très utiles pour reformatter des data, notamment lorsqu'elles sont fournies dans un format pour les humains et doivent être transformées dans un format pour les machines.

Voir : https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html

Voir également l'article Tidy Data.

In [ ]:
# exemple
s = df.groupby(['Continent', 'Country']).size()
s

In [ ]:
# exemple
s.unstack()

In [ ]:
# exemple
s.unstack('Continent')

In [ ]:
# exemple
tab = df.groupby('Continent').agg({'population': 'sum',
                                   'elevation': 'mean',
                                   'Country': ['min', 'max']})
tab

In [ ]:
# exemple
tab.stack()

In [ ]:
# exemple
tab.stack(0)

**Remarque** : la méthode `pivot_table()` peut être simulée avec un `groupby()` suivi d'un `unstack()`.

In [ ]:
# remarque pivot_table = groupby + unstack
df.pivot_table(values='Population', index='Country', columns='Continent', aggfunc='sum')

In [ ]:
# remarque pivot_table = groupby + unstack
df.groupby(['Country', 'Continent'])['Population'].sum().unstack()

Le `pivot_table()` est même un peu plus lent, mais plus facile à comprendre.

In [ ]:
# remarque pivot_table = groupby + unstack
%timeit df.pivot_table(values='Population', index='Country', columns='Continent', aggfunc='sum')

In [ ]:
# groupby + unstack
%timeit df.groupby(['Country', 'Continent'])['Population'].sum().unstack()

**Dataset n° 3**

United Nations (UNCTAD) with FDI inflows (Foreign direct investment), by region and economy from
1990 to 2018.

Ce fichier est en format large avec une colonne par année, facilement lisible par un humain.

In [ ]:
# UNCTAB dataset
df_un = pd.read_excel('WIR19_tab01.xlsx',
                      header=2,
                      nrows=240)
df_un = df_un.drop(0)
df_un['Region/economy'] = df_un['Region/economy'].apply(str.strip)
df_un.columns = ['Region/economy'] + [int(col) for col in df_un.columns[1:]]
df_un

On peut utiliser `stack()` + `reset_index()` pour le passer en format long, dit normalisé.

In [ ]:
# pour le passer au format long
# on peut utiliser stack + reset_index
df_un.set_index('Region/economy').stack().reset_index()

In [ ]:
# vérification de la longueur
len(df_un) * (df_un.shape[1] - 1)

La méthode `melt()` fait aussi ça très bien.

Elle prend comme arguments :
- la liste des colonnes associées à l'identité des enregistrements
- éventuellement la liste des colonnes associées aux différentes valeurs considérées

Elle génère un DataFrame normalisé avec les colonnes associées à l'identité des enregistrements, ainsi qu'une colonne "variable" correspondant aux noms des anciennes colonnes de valeurs et une colonne "value" correspondant aux valeurs des anciennes colonnes de valeurs.

In [ ]:
# ou bien tout simplement
# la méthode melt
tab = df_un.melt(id_vars=['Region/economy'])
tab

In [ ]:
# pour revenir pratiqument au DataFrame de départ
tab.set_index(['Region/economy', 'variable']).unstack().reset_index()

**Exercice n° 3**

Faites la jointure de `df_un` avec `df_pays` en utilisant la fonction `merge()` ou la méthode `join()`.

Comparez le nombre de lignes entre le résultat et `df_pays`. D'où vient le problème ?

Faire le mapping avec un dictionnaire de transcodification des noms des pays (voir le fichier `mapping.py`).

Repassez en format long avec la méthode `melt()`.

In [ ]:
from country_mapping import mapping
mapping